In [654]:
import cv2
import mediapipe as mp
import os
import csv

import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder

In [ ]:
# Initialiser MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode = True, max_num_hands = 2, min_detection_confidence = 0.5)
mp_drawing = mp.solutions.drawing_utils

# Fonction pour extraire les landmarks d'une image
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        # Récupérer les landmarks pour chaque main détectée
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Coordonnées X, Y, Z
            return landmarks
    print(f"Aucune main détectée pour {image_path}")
    return None

# Parcourir les dossiers et extraire les données
def process_dataset(dataset_path, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        # En-tête du fichier CSV
        header = ['label'] + [f'{coord}_{i}' for i in range(21) for coord in ['x', 'y', 'z']]
        writer.writerow(header)

        # Parcourir les dossiers
        for label in os.listdir(dataset_path):
            label_path = os.path.join(dataset_path, label)
            if os.path.isdir(label_path):
                for image_name in os.listdir(label_path):
                    image_path = os.path.join(label_path, image_name)
                    landmarks = extract_landmarks(image_path)
                    if landmarks:
                        writer.writerow([label] + landmarks)

# Chemins
dataset_path = "Train" 
output_csv = "landmarks.csv"

# Exécution
process_dataset(dataset_path, output_csv)
print(f"Données enregistrées dans {output_csv}")

In [ ]:
# Chemins pour l'échantillon de test
test_dataset_path = "Test"
test_output_csv = "test_landmarks.csv"

# Exécution pour l'échantillon de test
process_dataset(test_dataset_path, test_output_csv)
print(f"Données enregistrées dans {test_output_csv}")

In [656]:
# Charger les données d'entraînement obtenues précédemment
data_train = pd.read_csv("landmarks.csv")
print(data_train.head())  # Affiche les premières lignes pour vérifier le chargement

# Charger les données de test obtenues juste avant
data_test = pd.read_csv("test_landmarks.csv")
print(data_test.head())  # Affiche les premières lignes pour vérifier le chargement

   label       x_0       y_0           z_0       x_1       y_1       z_1  \
0  ENTER  0.313832  0.764376  2.474214e-07  0.358308  0.734033 -0.026857   
1  ENTER  0.650379  0.380406  2.110198e-07  0.603407  0.368063 -0.015487   
2  ENTER  0.663164  0.614228  1.384285e-07  0.619237  0.591525 -0.012178   
3  ENTER  0.338837  0.690563  3.175797e-07  0.293740  0.682436 -0.011694   
4  ENTER  0.716060  0.874130  3.947711e-07  0.661055  0.861623 -0.025433   

        x_2       y_2       z_2  ...      z_17      x_18      y_18      z_18  \
0  0.386194  0.675503 -0.038582  ... -0.034137  0.273438  0.577838 -0.052214   
1  0.568174  0.318842 -0.020643  ... -0.019743  0.653054  0.199351 -0.029746   
2  0.589870  0.539513 -0.016454  ... -0.018273  0.676231  0.433817 -0.025656   
3  0.258236  0.642395 -0.017841  ... -0.030567  0.360939  0.508950 -0.041258   
4  0.618185  0.820818 -0.042626  ... -0.042167  0.758279  0.681288 -0.056616   

       x_19      y_19      z_19      x_20      y_20      z_20 

In [658]:
data_train = data_train.dropna()  # Supprime les lignes avec des valeurs manquantes
print(f"Données d'entraînement après suppression : {data_train.shape}")

data_test = data_test.dropna()  # Supprime les lignes avec des valeurs manquantes
print(f"Données de test après suppression : {data_test.shape}")

Données d'entraînement après suppression : (5158, 64)
Données de test après suppression : (1833, 64)


In [660]:
# Etape de normalisation

scaler = MinMaxScaler()
features_train = data_train.iloc[:, 1:]  # Exclure la colonne des labels
features_test = data_test.iloc[:, 1:] # De même pour l'échantillon de test
normalized_features_train = scaler.fit_transform(features_train)
normalized_features_test = scaler.fit_transform(features_test)

# Remplacer les anciennes données par les données normalisées
data_train.iloc[:, 1:] = normalized_features_train
print(data_train.head())

data_test.iloc[:, 1:] = normalized_features_test
print(data_test.head())

   label       x_0       y_0       z_0       x_1       y_1       z_1  \
0  ENTER  0.329598  0.679027  0.780843  0.373168  0.673895  0.312188   
1  ENTER  0.662875  0.273039  0.747177  0.631281  0.284959  0.485181   
2  ENTER  0.675536  0.520269  0.680040  0.647952  0.522444  0.535528   
3  ENTER  0.354360  0.600981  0.845729  0.305172  0.619060  0.542895   
4  ENTER  0.727917  0.795074  0.917120  0.691990  0.809492  0.333856   

        x_2       y_2       z_2  ...      z_17      x_18      y_18      z_18  \
0  0.400932  0.646232  0.410156  ...  0.396241  0.287109  0.586272  0.419299   
1  0.590779  0.273370  0.583845  ...  0.524845  0.638448  0.199098  0.583771   
2  0.613413  0.504065  0.624400  ...  0.537979  0.659898  0.438945  0.613712   
3  0.267443  0.611621  0.610969  ...  0.428137  0.368093  0.515803  0.499499   
4  0.642952  0.798148  0.371002  ...  0.324492  0.735834  0.692096  0.387074   

       x_19      y_19      z_19      x_20      y_20      z_20  
0  0.293212  0.554767 

In [662]:
X_train = data_train.iloc[:, 1:]  # Toutes les colonnes sauf les labels
y_train = data_train.iloc[:, 0]   # La colonne des labels
print(f"Caractéristiques de l'échantillon d'entraînement (X_train) : {X_train.shape}, Labels (y_test) : {y_train.shape}")

X_test = data_test.iloc[:, 1:]  # Toutes les colonnes sauf les labels
y_test = data_test.iloc[:, 0]   # La colonne des labels
print(f"Caractéristiques de l'échantillon de test (X_test) : {X_test.shape}, Labels (y_test) : {y_test.shape}")

Caractéristiques de l'échantillon d'entraînement (X_train) : (5158, 63), Labels (y_test) : (5158,)
Caractéristiques de l'échantillon de test (X_test) : (1833, 63), Labels (y_test) : (1833,)


In [664]:
def add_noise(landmarks, noise_level = 0.01):
    noise = np.random.normal(0, noise_level, landmarks.shape)
    return landmarks + noise

def scale_landmarks(landmarks, scale_factor = 1.1):
    return landmarks * scale_factor

In [666]:
# Appliquer plusieurs augmentations
X_train_augmented = []

# Parcourir les lignes du DataFrame
for i in range(X_train.shape[0]):
    original = X_train.iloc[i, :]
    augmented = [
        add_noise(original),
        scale_landmarks(original)
    ]
    X_train_augmented.extend(augmented)

X_train_augmented = np.array(X_train_augmented)  # Convertir en tableau NumPy
print(X_train_augmented.shape)  # Vérifier la forme finale des données augmentées

(10316, 63)


In [668]:
X_train = X_train_augmented  # Remplacer les données originales par les augmentées
y_train = np.repeat(y_train, 2)  # Adapter les labels (chaque ligne originale a 2 augmentations)

In [670]:
print(f"Forme de X_train après augmentation : {X_train.shape}")
print(f"Forme de y_train après répétition des labels : {y_train.shape}")

Forme de X_train après augmentation : (10316, 63)
Forme de y_train après répétition des labels : (10316,)


In [672]:
# Encoder les labels en nombres entiers

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)  # Conversion des labels
y_test = label_encoder.transform(y_test)  # Même transformation pour les données de test

print(f"Labels encodés : {y_train}")  # Vérifier les labels transformés

Labels encodés : [0 0 0 ... 3 3 3]


In [674]:
# Conversion des données en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Créer des DataLoader pour charger les données par lots
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle=False)

In [676]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.Softmax(dim = 1)  # Activation pour la sortie
        )

    def forward(self, x):
        return self.model(x)

# Initialiser le modèle
input_size = X_train.shape[1]  # Nombre de caractéristiques
num_classes = len(np.unique(y_train))  # Nombre de classes
model = MLP(input_size, num_classes)
print(model)  # Afficher la structure du modèle

MLP(
  (model): Sequential(
    (0): Linear(in_features=63, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=4, bias=True)
    (5): Softmax(dim=1)
  )
)


In [678]:
criterion = nn.CrossEntropyLoss()  # Perte adaptée pour la classification multiclasses
optimizer = optim.Adam(model.parameters(), lr = 0.001)  # Optimiseur Adam

In [680]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Mode entraînement
    running_loss = 0.0

    for batch in train_loader:
        X_batch, y_batch = batch

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass et mise à jour
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Époque {epoch+1}/{num_epochs}, Perte : {running_loss/len(train_loader):.4f}")

Époque 1/10, Perte : 1.0034
Époque 2/10, Perte : 0.8417
Époque 3/10, Perte : 0.8242
Époque 4/10, Perte : 0.8157
Époque 5/10, Perte : 0.8108
Époque 6/10, Perte : 0.8081
Époque 7/10, Perte : 0.8051
Époque 8/10, Perte : 0.8046
Époque 9/10, Perte : 0.8001
Époque 10/10, Perte : 0.7980


In [682]:
model.eval()  # Mode évaluation
correct = 0
total = 0

with torch.no_grad():  # Pas de calcul des gradients
    for batch in test_loader:
        X_batch, y_batch = batch
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)  # Obtenir la classe prédite
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Précision sur l'ensemble de test : {100 * correct / total:.2f}%")

Précision sur l'ensemble de test : 77.03%


In [684]:
torch.save(model.state_dict(), "model.pth")

In [686]:
X_train.shape[1]

63

In [710]:
np.sum(y_train==3)/2

1436.0